In [ ]:
# Required Libararies
import numpy as np
import pygame
import sys
import math
import random
player=0
cpu=1
rows=6
columns=7
square_size=100
Radius=int(square_size/2 -5)
slide=4
player_piece=1
cpu_piece=2
empty=0

In [ ]:
#create board
def create_board():
    board=np.zeros((rows,columns))
    return board
def is_valid_choice(board,col):
    return board[rows-1][col]==0
def next_aval_row(board,col):
    for i in range(rows):
        if board[i][col]==0:
            return i
def drop_piece(board,col,row,peice):
    board[row][col]=peice
def winning_move(board,peice):
    #horizontal
    for c in range(columns-3):
        for r in range(rows):
            if board[r][c]==peice and board[r][c+1]==peice and board[r][c+2]==peice and board[r][c+3]==peice:
                    return True
    #vertical
    for c in range(columns):
        for r in range(rows-3):
            if board[r][c]==peice and board[r+1][c]==peice and board[r+2][c]==peice and board[r+3][c]==peice:
                    return True
    #diagonal
    for c in range(columns-3):
        for r in range(rows-3):
            if board[r][c]==peice and board[r+1][c+1]==peice and board[r+2][c+2]==peice and board[r+3][c+3]==peice:
                    return True
    #diagonal rows -ve col +ve
    for c in range(columns-3):
        for r in range(3,rows):
            if board[r][c]==peice and board[r-1][c+1]==peice and board[r-2][c+2]==peice and board[r-3][c+3]==peice:
                    return True
# heuristic value
def evaluation(board,piece):
    opposition=player_piece
    if piece==player_piece:
        opposition=cpu_piece
    score=0
    if board.count(piece)==4:
        score+=100
    elif board.count(piece)==3 and board.count(empty)==1:
        score+=5
    elif board.count(piece)==2 and board.count(empty)==2:
        score+=2
    if board.count(opposition)==3 and board.count(empty)==1:
        score-=4
    return score
def score_pos(board,piece):
    score=0
    in_centre=[int(i) for i in list(board[:,columns//2])]
    in_centre_count=in_centre.count(piece)
    score=in_centre_count*3
    for r in range(rows):
        row_list=[int(i) for i in list(board[r,:])]
        for c in range(columns-3):
            slide_col=row_list[c:c+slide]
            score+=evaluation(slide_col,piece)
    for c in range(columns):
        col_list=[int(i) for i in list(board[:,c])]
        for r in range(rows-3):
            slide_row=row_list[r:r+slide]
            score+=evaluation(slide_row,piece)
    for r in range(rows-3):
        for c in range(columns-3):
            slide_diagonal=[board[r+i,c+i] for i in range(slide)]
            score+=evaluation(slide_diagonal,piece)
    for r in range(rows-3):
        for c in range(columns-3):
            slide_diagonal_down=[board[r+3-i,c+i] for i in range(slide)]
            score+=evaluation(slide_diagonal_down,piece)
    return score
def dead_terminal(board):
    return winning_move(board,player_piece) or winning_move(board,cpu_piece) or len(get_valid(board))==0
def get_valid(board):
    valid_moves=[]
    for col in range(columns):
        if is_valid_choice(board,col):
            valid_moves.append(col)
    return valid_moves

def best_move(board,piece):
    valid_moves=get_valid(board)
    best_score=-10000000
    best_col=random.choice(valid_moves)
    for col in valid_moves:
        row=next_aval_row(board,col)
        copy_board=board.copy()
        drop_piece(copy_board,col,row,piece)
        score=score_pos(copy_board,piece)
        if score>best_score:
            best_score=score
            best_col=col
    return best_col
def alpha_beta_pruning(board,depth,alpha,beta,max_player):
    valid_location=get_valid(board)
    is_term=dead_terminal(board)
    if depth==0 or is_term:
        if is_term:
            if winning_move(board,cpu_piece):
                return (None,10000000)
            elif winning_move(board,player_piece):
                return(None,-10000000)
            else:
                return (None,0)
        else:
            return(None,score_pos(board,cpu_piece))
    if max_player:
        value=-math.inf
        col=random.choice(valid_location)
        for c in valid_location:
            row = next_aval_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, c, row, cpu_piece)
            new_score = alpha_beta_pruning(b_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                col = c
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return col, value
    else:
        value=math.inf
        col=random.choice(valid_location)
        for c in valid_location:
            row = next_aval_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, c, row, player_piece)
            new_score = alpha_beta_pruning(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                col = c
            beta = min(beta, value)
            if alpha >= beta:
                break
        return col, value
        
            

    
def py_display(board):
    for c in range(columns):
        for r in range(rows):
            pygame.draw.rect(screen,(0,0,255),(c*square_size,r*square_size+square_size,square_size,square_size))
            pygame.draw.circle(screen,(0,0,0),(c*square_size+int(square_size/2),r*square_size+square_size+int(square_size/2)),Radius)
    for c in range(columns):
        for r in range(rows):
            if board[r][c] == 1:
                pygame.draw.circle(screen, (255,0,0), (int(c*square_size+square_size/2), height-int(r*square_size+square_size/2)), Radius)
            elif board[r][c] == 2: 
                pygame.draw.circle(screen, (255,255,0), (int(c*square_size+square_size/2), height-int(r*square_size+square_size/2)), Radius)
    pygame.display.update() 

In [ ]:
board=create_board()

In [ ]:
print(board)

In [ ]:
game_over=False
turn=0
pygame.init()
width=columns*square_size
height=(rows+1)*square_size
size=(height,width)
screen=pygame.display.set_mode(size)
py_display(board)
pygame.display.update()
myfont = pygame.font.SysFont("monospace", 75)
turn=random.randint(player,cpu)
while not game_over:
    for event in pygame.event.get():
        if event.type==pygame.QUIT:
            sys.exit()
        if event.type == pygame.MOUSEMOTION:
            pygame.draw.rect(screen, (0,0,0), (0,0, width, square_size))
            posx = event.pos[0]
            if turn == player:
                pygame.draw.circle(screen, (255,0,0), (posx, int(square_size/2)), Radius)
        pygame.display.update()
        if event.type == pygame.MOUSEBUTTONDOWN:
            pygame.draw.rect(screen, (0,0,0), (0,0, width, square_size))
            if turn ==player:
                posx = event.pos[0]
                col = int(math.floor(posx/square_size))
                if is_valid_choice(board,col):
                    row=next_aval_row(board,col)
                    drop_piece(board,col,row,player_piece)
                    if winning_move(board,player_piece):
                        label = myfont.render("Player 1 wins!!", 1, (255,0,0))
                        screen.blit(label, (40,10))
                        game_over = True
                    turn+=1
                    turn=turn %2
                    print(board)
                    py_display(board)
    if turn==cpu and not game_over:
        col,alpha_beta=alpha_beta_pruning(board,6,-math.inf,math.inf,True)
        if is_valid_choice(board,col):
            row=next_aval_row(board,col)
            drop_piece(board,col,row,cpu_piece)
            if winning_move(board,cpu_piece):
                label = myfont.render("Player 2 wins!!", 1, (255,255,0))
                screen.blit(label, (40,10))
                game_over = True
            print(board)
            py_display(board)
            turn+=1
            turn=turn%2
    if game_over:
            pygame.time.wait(3000)
